In [ ]:
#Script Name: plot_multirmsd.ipynb
#Script Purpose: This script will plot the RMSD input from multiple cpptraj rmsd analyses,
## on top of one another, using boxcar averaging and the top10 color map.
## Requires user to assign the filename prefix and suffixes in cell 2. Out of the box, the prefixes
## are set for 3 replicates of a molecule named "26", plotting the _lig_min_nofit.dat file from each run.
## This naming is consistent with the MD RMSD analysis workflow available from the Rizzo Lab analysis tools.
#Author Name: John Bickel
#Affiliation: Rizzo Lab, Stony Brook University
#Create date: 2022/07/31
#Last edit: 2022/07/31 JBD or John Bickel/SBU

import pandas as pd                             #data set handling
import numpy as np                              #math functions
import matplotlib.pyplot as plt                 #plotting
import matplotlib.colors as col
from matplotlib.ticker import NullFormatter     #formatting for plot axes
from scipy.stats import gaussian_kde            #scatter plot density coloring
from scipy.optimize import curve_fit            #distribution fitting

def func(x, *params):
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        ctr = params[i]
        amp = params[i+1]
        wid = params[i+2]
        y = y + amp * np.exp(-((x - ctr)/ np.sqrt(2) / wid)**2)
    return y

def fit_histo(data_list, bin_num, guess):    
    y, x_ = np.histogram(data_list, bins = bin_num)
    x = []
    for i in range(0, len(x_) - 1):
        x.append(((x_[i + 1] - x_[i]) / 2) + x_[i])
    #guess = [-80, 20, 20, -110, 25, 20] [mean, height, stdev]
    popt, pcov = curve_fit(func, x, y, p0=guess)
    fit = func(x, *popt)
    fit_set = []
    for i in range(0, int(len(guess) / 3)):
        params = [popt[3 * i], popt[3 * i + 1], popt[3 * i + 2]]
        fit_set.append(func(x, *params))
    return [popt, x, fit_set]

In [ ]:
datadir="./md/zzz.raw_data"
filenames=["26_0","26_1","26_2"]
filenames=[f"{datadir}/{x}/{x}" for x in filenames]
file_suffix="_lig_min_nofit.dat"
#filename = 'rmsd1_minimized_dolu.agr'
fpns = 200 #frames per nanosecond, converts frame number to time

box_width = 100 #the size of the box will be 2N + 1 frames, with the box flanking the center frame by this many frames on either side

fit_guess = [1.5, 600, 1, 5, 300, 2] #[mean, peakheight, stdev, ... and repeat for each peak in fit]


nullfmt = NullFormatter()

# Plot multi-line plot + MultiHist

In [ ]:
# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width 

b = 0.2 # how tall the histograms are relative to the plot
rect_plot = [left, bottom, width, height]
rect_histy = [left_h, bottom, b, height]

# start with a rectangular Figure
plt.figure(1, figsize=(6, 4), dpi = 192)
plot_cmap=plt.cm.get_cmap("tab10",len(filenames)+1)
plot_cmap=[col.rgb2hex(plot_cmap(i)[:3]) for i in range(plot_cmap.N)]
axPlot = plt.axes(rect_plot)
axHisty = plt.axes(rect_histy)

# no y axis labels on histy, labels bad
axHisty.yaxis.set_major_formatter(nullfmt)

for counter,direc in enumerate(filenames):
    filename=f"{direc}{file_suffix}"

    df = pd.read_csv(filename, delim_whitespace=True) #dataframe read from the file csv, with delimited whitespaces
    df.columns = ['time','rmsd'] # sets the column headers for the dataframe
    df['time'] = df['time'] / fpns #fixes the time column to actual time, rather than frames


    #this is where the boxcar happens - nan values are so that the boxcar values x&y match the base
    rmsd = np.array(df['rmsd'])   #put it into a np array because pandas is slow af
    boxed = [np.nan] * box_width  #pad the results with nan values
    for i in range(box_width, rmsd.size - box_width):
        boxed.append(rmsd[i - box_width:i + box_width + 1].mean()) #boxcar, moving boxcar by 1 each time
    for i in range(0, box_width):
        boxed.append(np.nan)      #pad the ends with more nan
    df['boxed'] = boxed           #add the results to the DataFrame


    x = df['time']
    y = df['rmsd']

    binsy = 100   #adjust the number of bins for each histogram

    xlim = [0, df['time'].max()]   #axes limits
    ylim = [0, df['rmsd'].max() * 1.1]

    # if you want to draw dashed lines across the plot parallel to either axis
    xlines = [] 
    ylines = []

    #guesses for histogram fitting
    guessy = fit_guess


    
    # the rmsd plot:
    axPlot.plot(df['time'], df['rmsd'],  c=plot_cmap[counter], alpha=0.2)
    axPlot.plot(df['time'], df['boxed'], c=plot_cmap[counter], alpha=1.0)

    axPlot.set_xlim((xlim[0],xlim[1]))
    axPlot.set_ylim((ylim[0],ylim[1]))

    #labels
    axPlot.set_ylabel("RMSD (Å)",size=5)
    axPlot.set_xlabel("Time (ns)",size=5)
    axPlot.tick_params(axis='both',which='major',labelsize=6)
    axHisty.set_xlabel("Population",size=5)
    axHisty.tick_params(axis='x',which='major',labelsize=4)

    axHisty.hist(y, bins=binsy, orientation='horizontal', color='c', alpha=0.5)
    %config InlineBackend.figure_format ='retina'
plt.show()

# Plot Multi-line Plot, NoHist. 

In [ ]:
# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
bottom_h = left_h = left + width 

b = 0.2 # how tall the histograms are relative to the plot
rect_plot = [left, bottom, width, height]
rect_histy = [left_h, bottom, b, height]

# start with a rectangular Figure
plt.figure(1, figsize=(6, 4), dpi = 300)

plot_cmap=plt.cm.get_cmap("tab10",len(filenames)+1)
plot_cmap=[col.rgb2hex(plot_cmap(i)[:3]) for i in range(plot_cmap.N)]
axPlot = plt.axes(rect_plot)

for counter,direc in enumerate(filenames):
    filename=f"{direc}_lig_min_nofit.dat"

    df = pd.read_csv(filename, delim_whitespace=True) #dataframe read from the file csv, with delimited whitespaces
    df.columns = ['time','rmsd'] # sets the column headers for the dataframe
    df['time'] = df['time'] / fpns #fixes the time column to actual time, rather than frames


    #this is where the boxcar happens - nan values are so that the boxcar values x&y match the base
    rmsd = np.array(df['rmsd'])   #put it into a np array because pandas is slow af
    boxed = [np.nan] * box_width  #pad the results with nan values
    for i in range(box_width, rmsd.size - box_width):
        boxed.append(rmsd[i - box_width:i + box_width + 1].mean()) #boxcar, moving boxcar by 1 each time
    for i in range(0, box_width):
        boxed.append(np.nan)      #pad the ends with more nan
    df['boxed'] = boxed           #add the results to the DataFrame


    x = df['time']
    y = df['rmsd']

    binsy = 100   #adjust the number of bins for each histogram

    xlim = [0, df['time'].max()]   #axes limits
    ylim = [0, df['rmsd'].max() * 1.1]

    # if you want to draw dashed lines across the plot parallel to either axis
    xlines = [] 
    ylines = []

    #guesses for histogram fitting
    guessy = fit_guess

    # the rmsd plot:
    axPlot.plot(df['time'], df['rmsd'],  c=plot_cmap[counter], alpha=0.2)
    axPlot.plot(df['time'], df['boxed'], c=plot_cmap[counter], alpha=1.0)

    axPlot.set_xlim((xlim[0],xlim[1]))
    axPlot.set_ylim((ylim[0],ylim[1]))

    #labels
    axPlot.set_ylabel("RMSD (Å)",size=5)
    axPlot.set_xlabel("Time (ns)",size=5)
    axPlot.tick_params(axis='both',which='major',labelsize=6)
    %config InlineBackend.figure_format ='retina'
plt.show()

# Needs changing - plots only one histogram - currently nonfunctional

This is present 


In [ ]:
plt.figure(dpi=120)
plt.title('RMSD population density')
plt.ylabel('density')
plt.xlabel('RMSD (Å)')
plt.hist(df['rmsd'], color='c', bins=125, normed=True)
plt.xlim(0, df['rmsd'].max())
    
    
plt.show()